In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [2]:
raw = pd.read_csv( "../data/interim/lazada_review_clean_v2.0_2.csv")
slang_words = pd.read_csv("../data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("../data/external/slangword_ID.txt",
                        sep=":", header=None)
baku_words = pd.read_csv("../data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]
root_words = np.array(pd.read_csv("../data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.concat([slang_words, slang, baku_words])

In [3]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
def delete_suffix_nya(review):
    return re.sub("(?:nya|ny|y)[$|\s]"," ",review)
    
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review'] = raw['review'].apply(lambda x: x.split())

In [4]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [5]:
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [6]:
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x.split(",")))

In [7]:
raw.to_csv("../data/interim/lazada_review_cleaner_pure.csv", index=False)

In [8]:
stop_words = np.array(pd.read_csv("../data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)
neg_words = np.array(pd.read_csv("../data/external/negative_keyword_ID.txt",
                        sep="\n", header=None).values)
pos_words = np.array(pd.read_csv("../data/external/positive_keyword_ID.txt",
                        sep="\n", header=None).values)
booster_words = np.array(pd.read_csv("../data/external/boosterword_ID.txt",
                        sep="\n", header=None).values)
sentiment_words = np.concatenate((pos_words, neg_words, booster_words))
stop_words = np.reshape(stop_words, -1)

In [9]:
raw.head()
raw['review'] = raw['review'].apply(lambda x: x.split(","))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tvec_weights = vectorizer.fit_transform(raw['review'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=False)
domain_spesific_stop_words = np.array(weights_df['term'][:200].values)
stop_words = np.concatenate((domain_spesific_stop_words,stop_words[:,]))
stop_words = [word for word in stop_words if word not in sentiment_words]
stop_words = np.unique(stop_words)

In [11]:
def remove_stop_words(word_list):
    return [word for word in word_list if word not in stop_words]
raw['review'] = raw['review'].apply(remove_stop_words)
raw.head()

,rating,review,avg_word,word_count
0,1,"[sangat, lama, jauh, berbeda, internal, lebih,...",5.750000,20
1,1,"[pesananku, terkirim, baik, juta, ribu, enam, ...",5.408602,93
2,5,"[tidak, sia, sia, susah, payah, ikutan, naik, ...",5.166667,30
3,5,"[tujuh, gagal, padahal, tidak, menyangka, seni...",5.750000,28
4,5,"[kurang, setuju, memang, strategi, penasaran, ...",5.295082,61


In [12]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x))
raw['word_count'] = raw['review'].map(len)

In [13]:
raw.to_csv("../data/interim/lazada_review_cleaner_stop_removed.csv", index=False)